In [2]:
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install Bio

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gchir\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 10.7/10.7 MB 4.5 MB/s eta 0:00:00
     -------------------------------------- 341.8/341.8 KB 1.9 MB/s eta 0:00:00
     -------------------------------------- 502.3/502.3 KB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gchir\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gchir\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('sample_information_American_gut_microbiome.csv.txt', sep="\t", dtype=object)
print('Numero di campioni: ', data.shape[0], '\nInformazioni: ', data.shape[1]-1)

Numero di campioni:  29960 
Informazioni:  660


In [3]:
data.head()

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
0,10317.000001000,Not provided,false,false,"Diagnosed by a medical professional (doctor, p...",60s,64.0,64,true,Daily,...,Not provided,Never,Regularly (3-5 times/week),Not provided,NaN,Remained stable,52,kilograms,Never,Occasionally (1-2 times/week)
1,10317.000001001,Not provided,false,false,Not provided,50s,53.0,53.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,false,false,Not provided,50s,53.0,53.0,true,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,false,false,Not provided,40s,44.0,44.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,false,false,Not provided,60s,66.0,66.0,false,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided


In [4]:
data[data=='true']='Yes'

In [5]:
data[data=='false']='No'
data[data=='I do not have this condition']='No'

In [6]:
data['lactose'].value_counts()

lactose
No                 20422
Yes                 3802
Not applicable      3598
Not provided         773
LabControl test      681
Unspecified          594
not applicable        10
not collected          3
Name: count, dtype: int64

In [7]:
list(data.columns)

['sample_name',
 'acid_reflux',
 'acne_medication',
 'acne_medication_otc',
 'add_adhd',
 'age_cat',
 'age_corrected',
 'age_years',
 'alcohol_consumption',
 'alcohol_frequency',
 'alcohol_types',
 'alcohol_types_beercider',
 'alcohol_types_red_wine',
 'alcohol_types_sour_beers',
 'alcohol_types_spiritshard_alcohol',
 'alcohol_types_unspecified',
 'alcohol_types_white_wine',
 'allergic_to',
 'allergic_to_i_have_no_food_allergies_that_i_know_of',
 'allergic_to_other',
 'allergic_to_peanuts',
 'allergic_to_shellfish',
 'allergic_to_tree_nuts',
 'allergic_to_unspecified',
 'altitude',
 'alzheimers',
 'animal_age',
 'animal_free_text',
 'animal_gender',
 'animal_origin',
 'animal_type',
 'anonymized_name',
 'antibiotic_history',
 'appendix_removed',
 'artificial_sweeteners',
 'asd',
 'assigned_from_geo',
 'autoimmune',
 'birth_year',
 'bmi',
 'bmi_cat',
 'bmi_corrected',
 'body_habitat',
 'body_product',
 'body_site',
 'bowel_movement_frequency',
 'bowel_movement_quality',
 'breastmilk_for

In [8]:
data.specialized_diet_exclude_refined_sugars.value_counts()

specialized_diet_exclude_refined_sugars
Not provided       13188
No                 10029
Not applicable      3598
Yes                 2101
LabControl test      681
Unspecified          203
not applicable        10
not collected          3
Name: count, dtype: int64

In [9]:
data.sibo.value_counts()

sibo
No                                                                   18152
Not provided                                                          5580
Not applicable                                                        3598
LabControl test                                                        681
Unspecified                                                            660
Self-diagnosed                                                         494
Diagnosed by a medical professional (doctor, physician assistant)      445
Diagnosed by an alternative medicine practitioner                      260
not applicable                                                          10
not collected                                                            3
Name: count, dtype: int64

In [10]:
data = data.loc[data.body_site == "UBERON:feces",:]

In [11]:
cond_no_disease=(((data['acid_reflux']=='No') | (data['acid_reflux']=='Not provided')) &
(data['alcohol_frequency']!='Daily') &
(data['allergic_to_peanuts']== 'No') &
(data['allergic_to_shellfish']== 'No') &
(data['allergic_to_tree_nuts']=='No') &
(data['allergic_to_other']=='No') &
((data['autoimmune']=='No') | (data['autoimmune']=='Not provided')) &
((data['cancer']=='No') | (data['cancer']=='Not provided')) &
((data['cdiff']=='No') | (data['cdiff']=='Not provided')) &
((data['diabetes']=='No') | (data['diabetes']=='Not provided')) &
((data['gluten']=='No') | (data['gluten']=='Not provided')) &
((data['ibd']=='No') | (data['ibd']=='Not provided')) &
((data['ibs']=='No') | (data['ibs']=='Not provided')) &
((data['kidney_disease']=='No') | (data['kidney_disease']=='Not provided')) &
((data['lactose']=='No') | (data['lactose']=='Not provided')) &
((data['liver_disease']=='No') | (data['liver_disease']=='Not provided')) &
((data['mental_illness_type_anorexia_nervosa']=='No') | (data['mental_illness_type_anorexia_nervosa']=='Not provided')) &
((data['mental_illness_type_bulimia_nervosa']=='No') | (data['mental_illness_type_bulimia_nervosa']=='Not provided')) &
((data['sibo']=='No') | (data['sibo']=='Not provided')))

In [12]:
data_no_disease = data.loc[cond_no_disease, :]

In [13]:
data_no_disease

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
1,10317.000001001,Not provided,No,No,Not provided,50s,53.0,53.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,No,No,Not provided,50s,53.0,53.0,Yes,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,No,No,Not provided,40s,44.0,44.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,No,No,Not provided,60s,66.0,66.0,No,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided
6,10317.000001018,Not provided,No,Yes,Not provided,20s,28.0,28.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,77.0,kilograms,Not provided,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26302,10317.000115230,No,No,No,No,20s,27,27,Yes,Regularly (3-5 times/week),...,LabControl test,Rarely (a few times/month),Occasionally (1-2 times/week),Regularly (3-5 times/week),NaN,Decreased more than 10 pounds,96,kilograms,Daily,Occasionally (1-2 times/week)
26333,10317.000115677,No,No,No,No,Unspecified,Unspecified,5,No,Never,...,LabControl test,Never,Daily,Never,NaN,Remained stable,Unspecified,kilograms,Never,Never
26334,10317.000115686,No,No,No,No,Unspecified,Unspecified,1,No,Never,...,LabControl test,Never,Daily,Never,NaN,Remained stable,12,kilograms,Occasionally (1-2 times/week),Regularly (3-5 times/week)
26345,10317.000116371,No,No,No,No,60s,64,64,Yes,Rarely (a few times/month),...,LabControl test,Never,Never,Rarely (a few times/month),NaN,Remained stable,99,kilograms,Regularly (3-5 times/week),Occasionally (1-2 times/week)


In [14]:
column_filt = ["sample_name",
               "age_years",
               "age_cat",
               "sex",
               "specialized_diet_exclude_dairy",
               "specialized_diet_exclude_refined_sugars",
               "body_site",
               "specialized_diet_i_do_not_eat_a_specialized_diet",
               "specialized_diet_paleodiet_or_primal_diet"]

In [15]:
data_f = data_no_disease.loc[:,column_filt]

In [16]:
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "false","specialized_diet_i_do_not_eat_a_specialized_diet"] = "No"
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "true","specialized_diet_i_do_not_eat_a_specialized_diet"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_dairy == "false","specialized_diet_exclude_dairy"] = "No"
data_f.loc[data_f.specialized_diet_exclude_dairy == "true","specialized_diet_exclude_dairy"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "false","specialized_diet_exclude_refined_sugars"] = "No"
data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "true","specialized_diet_exclude_refined_sugars"] = "Yes"

In [17]:
data_diet = data_f.loc[(data_f.specialized_diet_exclude_dairy == "No") |
                        (data_f.specialized_diet_exclude_dairy == "Yes") |
                        (data_f.specialized_diet_exclude_refined_sugars == "No") |
                        (data_f.specialized_diet_exclude_refined_sugars == "Yes")|
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No") |
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "Yes"),:]

In [18]:
data_diet

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
74,10317.000001166,47,40s,female,No,No,UBERON:feces,Yes,No
104,10317.000001226,67,60s,male,No,No,UBERON:feces,No,No
141,10317.000001300,50,50s,female,No,No,UBERON:feces,No,No
555,10317.000002224,36,30s,male,No,No,UBERON:feces,No,No
887,10317.000003009,63,60s,male,No,No,UBERON:feces,No,No
...,...,...,...,...,...,...,...,...,...
26302,10317.000115230,27,20s,male,No,Yes,UBERON:feces,Yes,No
26333,10317.000115677,5,Unspecified,male,No,No,UBERON:feces,Yes,No
26334,10317.000115686,1,Unspecified,female,No,No,UBERON:feces,Yes,No
26345,10317.000116371,64,60s,male,No,No,UBERON:feces,Yes,No


In [19]:
pd.crosstab(data_diet.specialized_diet_paleodiet_or_primal_diet	,
            data_diet.specialized_diet_exclude_refined_sugars, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_refined_sugars,No,Yes,All
specialized_diet_paleodiet_or_primal_diet,,,
No,2958,287,3245
Yes,16,19,35
All,2974,306,3280


In [20]:
pd.crosstab(data_diet.sex,
            data_diet.specialized_diet_exclude_dairy, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_dairy,No,Yes,All
sex,,,
Not provided,46.0,NaN,46
female,1524.0,65.0,1589
male,1572.0,66.0,1638
other,2.0,NaN,2
unspecified,5.0,NaN,5
All,3149.0,131.0,3280


In [21]:
pd.crosstab(data_diet.age_cat,
            data_diet.specialized_diet_exclude_dairy, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_dairy,No,Yes,All
age_cat,,,
20s,400,19,419
30s,628,20,648
40s,546,23,569
50s,527,25,552
60s,378,13,391
70+,130,5,135
Not provided,102,4,106
Unspecified,124,14,138
child,190,4,194


In [22]:
list(data_diet.age_cat.unique())

['40s',
 '60s',
 '50s',
 '30s',
 'Not provided',
 '20s',
 'teen',
 'Unspecified',
 'child',
 '70+']

In [23]:
age_categ = ['20s', '30s', '40s', '50s']

In [24]:
data_diet_f = data_diet[((data_diet['age_cat'] != 'Unspecified') &
                      (data_diet['age_years'] != 'Not provided') &
                      (data_diet['age_cat'] != 'child') &
                      (data_diet['age_cat'] != 'teen'))  &
                      ((data_diet.sex == 'female') |
                        (data_diet.sex == 'male')) &
                        ((data_diet['age_cat'] != '60s') | (data_diet['age_cat'] != '70s'))]

In [25]:
data_diet_f.describe

<bound method NDFrame.describe of            sample_name age_years age_cat     sex   
74     10317.000001166        47     40s  female  \
104    10317.000001226        67     60s    male   
141    10317.000001300        50     50s  female   
555    10317.000002224        36     30s    male   
887    10317.000003009        63     60s    male   
...                ...       ...     ...     ...   
26291  10317.000115216        63     60s  female   
26293  10317.000115219        62     60s  female   
26302  10317.000115230        27     20s    male   
26345  10317.000116371        64     60s    male   
26349  10317.000116379        56     50s    male   

      specialized_diet_exclude_dairy specialized_diet_exclude_refined_sugars   
74                                No                                      No  \
104                               No                                      No   
141                               No                                      No   
555                  

In [26]:
data_diet_f.head(5)

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
74,10317.000001166,47,40s,female,No,No,UBERON:feces,Yes,No
104,10317.000001226,67,60s,male,No,No,UBERON:feces,No,No
141,10317.000001300,50,50s,female,No,No,UBERON:feces,No,No
555,10317.000002224,36,30s,male,No,No,UBERON:feces,No,No
887,10317.000003009,63,60s,male,No,No,UBERON:feces,No,No


In [27]:
data_diet_f.specialized_diet_exclude_refined_sugars.value_counts()

specialized_diet_exclude_refined_sugars
No     2464
Yes     279
Name: count, dtype: int64

In [28]:
data_diet_f.age_years = data_diet_f.age_years.astype(int)

C:\Users\gchir\AppData\Local\Temp\ipykernel_6308\59878515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_diet_f.age_years = data_diet_f.age_years.astype(int)


In [29]:
np.sum((data_diet_f.specialized_diet_exclude_dairy == "No") & 
                            (data_diet_f.specialized_diet_exclude_refined_sugars == "Yes") &
                            (data_diet_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No")) 

214

In [30]:
refined_sugar_sample = data_diet_f.loc[data_diet_f['specialized_diet_exclude_refined_sugars'] == 'Yes']

In [31]:
refined_sugar_sample

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
6610,10317.000023139,69,60s,female,No,Yes,UBERON:feces,No,No
6809,10317.000023608,59,50s,male,No,Yes,UBERON:feces,No,No
8522,10317.000030449,27,20s,male,No,Yes,UBERON:feces,No,Yes
9440,10317.000033280,60,60s,female,No,Yes,UBERON:feces,No,No
9804,10317.000035367,41,40s,female,No,Yes,UBERON:feces,No,Yes
...,...,...,...,...,...,...,...,...,...
25831,10317.000113033,38,30s,male,No,Yes,UBERON:feces,Yes,No
25834,10317.000113037,49,40s,female,No,Yes,UBERON:feces,No,No
26079,10317.000114439,55,50s,male,No,Yes,UBERON:feces,No,No
26116,10317.000114499,22,20s,male,No,Yes,UBERON:feces,No,No


FastQ research:

In [32]:
from Bio import Entrez
Entrez.email = 'grusso@campus.unimib.it'

import xml.etree.ElementTree as ET
import os

In [40]:
def from_samplenames_to_fastqa(sample_names_column, download = False):
    
    # Inizializzo le liste vuote che poi riempirò durante il ciclo
    sample_names = []
    biosample_ids = []
    sra_ids = []
    sra_names = []
    sra_search_res = []
    run_ids = []    
    names_without_biosamples = []
    names_without_runs = []
    
    tot = len(sample_names_column)
    for i in range(len(sample_names_column)):
        
        sample = sample_names_column.iloc[i]
        print('\n', i)
        print('Sample name: ', sample)

        # Da sample name a biosample id:
        handleSce = Entrez.esearch(db='biosample', term=sample)
        recordSce = Entrez.read(handleSce)
        print('--Numero di BiosampleIDs trovati: ', len(recordSce['IdList']))

        if len(recordSce['IdList']) == 0:
            names_without_biosamples.append(sample)
            
        else:
            for b in recordSce['IdList']:
            
                print('----BioSampleID: ', b)

                # Da biosample id a SRA id
                try:
                    handleID = Entrez.efetch(db='biosample', id=b, retmode='xml')
                    root = ET.fromstring(handleID.read())
                    identifier = root.findall('.//BioSample//Ids//Id')
                except:
                    print('zio pera')
                    continue
                

                for i in identifier:
                    if i.attrib['db'] == 'SRA':
                        sraId = i.text

                        print('------SRA id: ', sraId)

                        # Da SRAid a RUNid
                        handleSra = Entrez.esearch(db='sra', term=sraId,retmode="xml")
                        resultsSra = Entrez.read(handleSra)["IdList"]



                        print('--------SRA search results: ', resultsSra)

                        for s in resultsSra:

                            handlesngSraId = Entrez.efetch(db='sra', id=s)
                            root = ET.fromstring(handlesngSraId.read())   
                            identifier = root.find('.//EXPERIMENT_PACKAGE//RUN_SET//RUN')
                            instrument = root.find('.//EXPERIMENT_PACKAGE//EXPERIMENT//PLATFORM//ILLUMINA//INSTRUMENT_MODEL')
                            
                            
                            print(instrument.text)
                            if (identifier != None) & (instrument.text == "Illumina MiSeq"):
                                runId = identifier.attrib['accession']

                                print('----------Run ID: ', runId)
                                
                                sample_names.append(sample)
                                biosample_ids.append(b)
                                sra_ids.append(sraId)
                                sra_search_res.append(s)
                                run_ids.append(runId)

                                # Download FASQ e FASTA dei RUNid
                                if download:

                                    print('------------Downloading FASTQ...')
                                    command2 = 'fastq-dump --readids' + runId
                                    os.system(command2)
                               
                            else:
                                names_without_runs.append(sample)

    dict = {'sample_names': sample_names, 
            'biosample_ids': biosample_ids, 
            'sra_ids': sra_ids,
            'sra_search_res': sra_search_res,
            'run_ids': run_ids,
            'names_without_biosamples': names_without_biosamples,
            'names_without_runs': names_without_runs}  

    return dict

# Funzione per trasformare il dictionary in un dataframe
def extract_ids_info_to_df(d):
    
    

    return df

# Funzione per salvare i risultati in un (anzi 3) csv
def save_result_ids(d, name, i):

    df = extract_ids_info_to_df(d)
    
    

In [42]:
tmp = from_samplenames_to_fastqa(refined_sugar_sample.sample_name, download = True)


 0
Sample name:  10317.000023139
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  7353729
zio pera

 1
Sample name:  10317.000023608
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  8568870
------SRA id:  ERS2203044
--------SRA search results:  ['5125216']
Illumina MiSeq
----------Run ID:  ERR2303818
------------Downloading FASTQ...

 2
Sample name:  10317.000030449
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  9541103
------SRA id:  ERS2483074
--------SRA search results:  ['5840380']
Illumina MiSeq
----------Run ID:  ERR2579642
------------Downloading FASTQ...

 3
Sample name:  10317.000033280
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  8577201
------SRA id:  ERS2213544
--------SRA search results:  ['5144891']
Illumina MiSeq
----------Run ID:  ERR2313963
------------Downloading FASTQ...

 4
Sample name:  10317.000035367


In [125]:
len(np.array(tmp['names_without_biosamples']))

9